## Code Description:

This is just ot predict number of alerts, nothing more.
I might be better to use number of alerts as a weight while predicting flux?

In [1]:
import os,sys
import numpy as np
import pandas as pd
import pyLCR
from astropy.io import fits
import pickle
import datetime
from codes_from_pylcr import *

import joblib
import glob
import matplotlib.pyplot as plt

# For training:
import xgboost as xgb

import glob


The Fermi-LAT Light Curve Repository Toolkit v0.1.0
Support Contact: Daniel Kocevski (daniel.kocevski@nasa.gov)


In [2]:
os.chdir('/Users/aadesai1/Desktop/In_use/ML_work/Fermi_amego_alert_project/Fermi_Sample/')

In [3]:
flux_factor_for_alert=50

In [4]:
num_alerts_files = glob.glob(f'alerts_per_year_flux_factor_{flux_factor_for_alert}/num_alerts_per_year_file_*')
alerts_per_bin_files = glob.glob(f'alerts_flux_data_per_year_file_{flux_factor_for_alert}/num_alerts_per_year_file_*')

In [5]:
src_names=[]
for path_name in num_alerts_files:
    src_names.append(path_name[-21:][:-4])

# Starting prediction!

In [6]:
# Make new feature files or sue old ones:
make_feature_files=True

# Set Debug Flag
debug_this=False

xvals = np.arange(1,19,1)

seed=1234
if os.path.isdir(f'seed_{seed}/')!=True:
    os.mkdir(f'seed_{seed}/')

In [7]:
for src_index in range(len(src_names)):
    if make_feature_files==False:
        continue
    #if src_names[src_index]!="4FGL_J0118.3-6008":
    #    continue
    
    all_times=xvals
    all_num_alerts=np.load(num_alerts_files[src_index])
    
    df = pd.DataFrame.from_records(np.transpose([all_times,np.asarray(all_num_alerts)]), columns=['#MJD','n_alerts'])
    df.loc[-1] = [0, 0] 
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index
    df.set_index('#MJD', inplace=True) #This replaces the index with the MJD value
    
    if debug_this==True:
        # Display the DataFrame
        df.plot(figsize=(10,5))
    
    batch_df = pd.DataFrame()
    #print(df)
    # Create Lagging features
    batch_df['lag_1'] = df['n_alerts'].shift(1) # Prediction 1 Use numdays: -1 day 0-1 = 1

    batch_df['lag_2'] = df['n_alerts'].shift(2) # Prediction 2 Use numdays: -2 day 0-2 = 2
    batch_df['lag_3'] = df['n_alerts'].shift(3) # Prediction 3 Use numdays: -3 day 0-4 = 3

    #batch_df['lag_11'] = df['n_alerts'].shift(11) # Prediction 3 Use numdays: -14 day 3-14 = 11
    #batch_df['lag_12'] = df['n_alerts'].shift(12) # Prediction 2 Use numdays: -14 day 2-14 = 12
    #batch_df['lag_13'] = df['n_alerts'].shift(13) # Prediction 1 Use numdays: -14 day 1-14 = 13


    # Rolling statistics
    batch_df['rolling_mean_3'] = df['n_alerts'].rolling(window=3).mean().round(2)
    batch_df['rolling_std_3'] = df['n_alerts'].rolling(window=3).std().round(2)

    # Inspect target variable - it's actually 3!

    # Lagging target variable
    batch_df['target_1d'] = df['n_alerts'].shift(-1) # Next day
    batch_df['target_2d'] = df['n_alerts'].shift(-2) # Second-next day
    batch_df['target_3d'] = df['n_alerts'].shift(-3) # Third-next day

    # Run predictions with all three models

    #print(batch_df)
    predictions_ls = [] # list to store predictions
    batch_df = batch_df.dropna()

    corr = batch_df.corr()
    corr.style.background_gradient(cmap='coolwarm')
    features_df = batch_df[["lag_1", "lag_2", "lag_3",  "rolling_mean_3", "rolling_std_3"]] 
    #features_df = batch_df[["lag_1", "lag_2", "rolling_mean_7", "rolling_std_7"]] 
    # Directory for feature store
    directory = f'seed_{seed}/feature_store'
    if not os.path.exists(directory):
        os.makedirs(directory)


    id = "v1"
    features_df.to_csv(f'{directory}/{src_names[src_index]}_features_{id}.csv')
    #targets_df = batch_df[["target_1d", "target_2d", "target_3d"]] 
    targets_df = batch_df[["target_1d"]] 
    targets_df.to_csv(f'{directory}/{src_names[src_index]}_targets_{id}.csv')
    

In [8]:
# Check correlation matrix. 
# Ideally we want low correlation between features, but high correlation between features and target
# features are lgs and target are target_nd 
# This is one of the things you need to do to improve the model, slice the data so there are more better correlations (cloer to 1)

#corr = batch_df.corr()
#corr.style.background_gradient(cmap='coolwarm')

In [9]:
# Make new feature files or sue old ones:
make_training_files=True

In [39]:
def train_model(fermi_src,directory,id):

    
    X_train = pd.read_csv(f'{directory}/{fermi_src}_features_{id}.csv')
    X_train.set_index('#MJD', inplace=True) #This replaces the index with the MJD value
    leave_number_indices = len(X_train)-2
    print(len(X_train))
    X_train_mod = X_train.head(leave_number_indices) # save some rows for later
    X_train_mod

    targets_df = pd.read_csv(f'{directory}/{fermi_src}_targets_{id}.csv')
    targets_df.set_index('#MJD', inplace=True) #This replaces the index with the MJD value
    targets_df_mod = targets_df.head(leave_number_indices) # save some rows for later 
    targets_df_mod

    # turn into a list
    y_train_ls= []
    for target in targets_df_mod.columns:
      y_train_ls.append(targets_df_mod[target])

    # Loop through targets - train one model for each target variable
    cv_results_ls = []
    for y_train in y_train_ls:
        
      # Prepare the DMatrix which is required by XGBoost
      dtrain = xgb.DMatrix(data=X_train_mod, label=y_train)

      # Define XGBoost parameters
      params = {
        #'max_depth': 6,
        #'min_child_weight': 1,
        'eta': 0.01,
        #'subsample': 1,
        #'colsample_bytree': 1,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse'
      }

      # Perform cross-validation
      cv_results = xgb.cv(
          params=params,
          dtrain=dtrain,
          #num_boost_round=100,#
          num_boost_round=100, #how often xgboost will try to build a better model
          nfold = 10,
          early_stopping_rounds=20, #will stop building models if there is no gain in accuracy
          metrics='rmse',
          as_pandas=True,
          seed=seed
      )
      cv_results_ls.append(cv_results)
      # Show the last mean RMSE as a measure of final performance
      print(f"Last mean RMSE: {cv_results['test-rmse-mean'][-1:]}")

    #print(cv_results['test-rmse-mean'])
    if debug_this==True:
      plt.figure(figsize=(10, 6))

      for idx, cv_results in enumerate(cv_results_ls):
          plt.plot(cv_results['test-rmse-mean'], label=f'CV Run {idx+1}')
          print(cv_results['test-rmse-mean'])

      plt.title('Test RMSE Through Iterations')
      plt.xlabel('Number of Boosting Rounds')
      plt.ylabel('Mean RMSE')
      plt.legend()
      plt.show()

    # List to store the final models
    final_models_ls = []

    # Use the same params here you used in cross-validation
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse'
      }

    # Iterate through each set of cross-validation results and corresponding training targets
    for cv_results, y_train in zip(cv_results_ls, y_train_ls):
        # Determine the optimal number of boosting rounds from cross-validation results
        optimal_boost_rounds = cv_results['test-rmse-mean'].idxmin() + 1

        # Initialize the XGBoost regressor with determined parameters
        final_model = xgb.XGBRegressor(
            n_estimators=optimal_boost_rounds,
            **params
        )

        # Train the model on the full training dataset
        final_model.fit(X_train_mod, y_train)

        # Append the trained model to the list
        final_models_ls.append(final_model)


    if os.path.isdir(f'seed_{seed}/models')!=True:
        os.mkdir(f'seed_{seed}/models')
    #os.mkdir('models')
    # Save each model with a unique filename
    for idx, final_model in enumerate(final_models_ls):
        filename = f'seed_{seed}/models/{fermi_src}_batch_demand_forecaster_model_{idx+1}.pkl'  # Create a unique filename for each model
        joblib.dump(final_model, filename)
    del dtrain,cv_results
    print("Ran!")
    return

In [40]:
for src_index in range(len(src_names)):
    #if src_names[src_index]!="4FGL_J0118.3-6008":
    #    continue
    if make_training_files==False:
        continue
    id = "v1"
    a = train_model(src_names[src_index],directory,id)


13
Last mean RMSE: 0    1.144675
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 78    0.567327
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 0    2.589671
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 3    1.101901
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 99    8.700494
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 1    1.540152
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 36    0.961267
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 0    12.993348
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 32    2.582614
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 0    1.10557
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 0    0.427383
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 59    2.560959
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 99    8.980327
Name: test-rmse-mean, dtype: float64
Ran!
13
Last mean RMSE: 

In [41]:
#targets_df = pd.read_csv(f'{directory}/{src_names[0]}_targets_{id}.csv')
#targets_df.set_index('#MJD', inplace=True) #This replaces the index with the MJD value
#targets_df_mod = targets_df.head(10) # save some rows for later 
#targets_df_mod
#len(targets_df)


# Now we Predict!

In [42]:
import matplotlib as mpl

font = {'size'   : 25}
mpl.rc('font', **font) 

In [43]:
if os.path.isdir(f'seed_{seed}/predictions')!=True:
    os.mkdir(f'seed_{seed}/predictions')


In [44]:
def make_plot(src_names,directory,id_val,target_val,predicted_val,example_source="none"):
    for src_index in range(len(src_names)):
        #if src_names[src_index]!="4FGL_J0118.3-6008":
        #    continue
    
        
        predictions_ls = [] # list to store predictions
        if example_source!="none":
            if example_source!=src_names[src_index]:
                continue

        features_df = pd.read_csv(f'{directory}/{src_names[src_index]}_features_{id_val}.csv')
        features_df.set_index('#MJD', inplace=True) #This replaces the index with the MJD value
        leave_number_indices=leave_number_indices = len(features_df)-2
        
        #for idx in range(1,4):  # Assuming there are 3 models
        for idx in [1]:  # Assuming there is 1 model
            filename = f'seed_{seed}/models/{src_names[src_index]}_batch_demand_forecaster_model_{idx}.pkl'  # Create a unique filename for each model
            model = joblib.load(filename)
            prediction = model.predict(features_df)
            predictions_ls.append(prediction)
        # Let's compare to the true values!

        targets_df = pd.read_csv(f'{directory}/{src_names[src_index]}_targets_{id_val}.csv')
        targets_df.set_index('#MJD', inplace=True) #This replaces the index with the MJD value

        predictions_df = pd.DataFrame({f'{predicted_val}': predictions_ls[i] for i in range(len(predictions_ls))})
        predictions_df.index = features_df.index
        result_df = predictions_df.join(targets_df, how = "inner")
        #print(result_df)
    
        fig = plt.figure(figsize=(12,7),dpi=200)
        ax = plt.subplot()

        ax.plot(result_df.index.array,result_df[target_val].array,"-",color="grey",linewidth=3,label="Original Data")
        ax.plot(result_df.index.array,result_df[predicted_val].array,":",color="red",linewidth=2,label="Training+Predicted Data")
        ax.plot([result_df.index.array[leave_number_indices],result_df.index.array[leave_number_indices]],[-0.05,10.15],"--",color="blue")  #For example agnstorm

        plt.ticklabel_format(axis='both', style='plain', scilimits=(4,9))

    
        #plt.text(result_df.index.array[int(-1*leave_number_indices/2)],9.5,"Training")  #For  example
        #plt.text(result_df.index.array[leave_number_indices+3],9.5,"Prediction") #For  example

        plt.xlabel("Time (MJD)")
        plt.ylabel("Count rate")
    
        plt.ylim(-0.05,10.15) #For  example
        #plt.xlim(result_df.index.array[0]-0.5,result_df.index.array[-1]+11) #For   example
        #ax.set_xticklabels([str(int(x)) for x in ax.get_xticks()])
        plt.legend()

        plt.savefig(f'seed_{seed}/predictions/{src_names[src_index]}_{target_val}.pdf',bbox_inches="tight")

        if example_source=="none":
            plt.close()
    


In [45]:
# Making all points for 1 day targets:
make_plot(src_names,directory,id,'target_1d','predicted_1d')


In [ ]:
# Making all points for 2 day targets:
make_plot(src_names,directory,id,'target_2d','predicted_2d')


In [82]:
# Making all points for 3 day targets:
make_plot(src_names,directory,id,'target_3d','predicted_3d')


## Example
To check an example, we will use one of the sources for which there exists a good amount of data.

In [ ]:
# Making an example plot using 3 day targets for one source:
make_plot(saved_fermi_lcs,directory,id,'target_3d','predicted_3d',example_source="4FGL_J1058.6-8003")

